## ReMKiT1D input generator - logical boundary condition test

This example tests the logical boundary condition by looking at the sheath heat transmission coefficient for a Maxwellian distribution

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn


### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27 #atomic mass unit
ionMass = 2.014*amu # deuterium mass
epsilon0 = 8.854188e-12 #vacuum permittivity 
heavySpeciesMass = 2.014 #in amus


### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_lbc_test/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 1 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

#### PETSc

Default settings are used here.

#### HDF5

No input HDF5 file is used here and all variables are outputted by default using the wrapper class.

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

timeNorm = skn.collTimeei(rk.normalization["eVTemperature"],rk.normalization["density"],rk.normalization["referenceIonZ"])

### Grid setup

In [ ]:
xGrid = np.ones(4) 
L = sum(xGrid)
dv0 = 0.05
cv = 1.025
vGrid = [dv0/2]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 1 
gridObj = Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True)

In [ ]:
# Add the grid to the wrapper
rk.grid=gridObj

### Set temperature derivation option and add electron species

In [ ]:
rk.setStandardTextbookOptions([0]) 

rk.addSpecies("e",0)
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0)

### Add custom derivations

These are the extrapolation derivations for the distribution function and for fluid variables and the target current derivations.

In [ ]:
rk.addCustomDerivation("lbcRightExt",sc.distScalingExtrapolationDerivation(True,True))
rk.addCustomDerivation("lbcLeftExt",sc.distScalingExtrapolationDerivation(True,True,True))

rk.addCustomDerivation("logLinExtrapRight",sc.boundedExtrapolationDerivation(sc.linLogExtrapolation()))
rk.addCustomDerivation("logLinExtrapLeft",sc.boundedExtrapolationDerivation(sc.linLogExtrapolation(leftBoundary=True),ignoreLowerBound=True))

rk.addCustomDerivation("currentDeriv",sc.simpleDerivation(multConst=np.sqrt(elMass/ionMass),varPowers=[1.0]))
rk.addCustomDerivation("currentDerivLeft",sc.simpleDerivation(multConst=-np.sqrt(elMass/ionMass),varPowers=[1.0]))


### Variables

In [ ]:
n = 1.0 + 0.001*np.sin(2*np.pi*gridObj.xGrid/L)
T = np.ones(gridObj.numX())
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (np.pi*T[i])**(-1.5) * n[i]* np.exp(-gridObj.vGrid**2/T[i])

numerical_dens = gridObj.velocityMoment(f,0,1)
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = n[i] *f[i,gridObj.getH(0)-1,:]/numerical_dens[i]

rk.addVarAndDual("f",f,isDistribution=True,isCommunicated=True)
rk.addVarAndDual("n",n,units='$10^{19} m^{-3}$',isDerived=True,derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVar("nb_right",isDerived=True,isScalar=True,units='$10^{19} m^{-3}$',derivationRule=sc.derivationRule("logLinExtrapRight",["n"]))
rk.addVar("nb_left",isDerived=True,isScalar=True,units='$10^{19} m^{-3}$',derivationRule=sc.derivationRule("logLinExtrapLeft",["n"]))

rk.addVar("ionCurrent_right",isDerived=True,isScalar=True,derivationRule=sc.derivationRule("currentDeriv",["nb_right"]))
rk.addVar("ionCurrent_left",isDerived=True,isScalar=True,derivationRule=sc.derivationRule("currentDerivLeft",["nb_left"]))

# Boundary property value containers
rk.addVar("gamma_right",isDerived=True,isScalar=True)
rk.addVar("potential_right",isDerived=True,isScalar=True)
rk.addVar("temp_right",isDerived=True,isScalar=True)

rk.addVar("gamma_left",isDerived=True,isScalar=True)
rk.addVar("potential_left",isDerived=True,isScalar=True)
rk.addVar("temp_left",isDerived=True,isScalar=True)

rk.addVar("time",isScalar=True,isDerived=True)

### LBC models

In [ ]:
cm.addLBCModel("lbc_right","f",rk,sc.derivationRule("lbcRightExt",["f","n","n_dual","nb_right"]),
                "ionCurrent_right",evolvedHarmonics=[1])

cm.addLBCModel("lbc_left","f",rk,sc.derivationRule("lbcLeftExt",["f","n","n_dual","nb_left"]),
                "ionCurrent_left",evolvedHarmonics=[1],leftBoundary=True)

### Manipulators

Adding manipulators to extract boundary properties from modelbound data

In [ ]:
rk.addManipulator("gammaExtRight",sc.extractorManipulator("lbc_right","gamma","gamma_right"))
rk.addManipulator("potentialExtRight",sc.extractorManipulator("lbc_right","potential","potential_right"))
rk.addManipulator("tempExtRight",sc.extractorManipulator("lbc_right","shTemp","temp_right"))

rk.addManipulator("gammaExtLeft",sc.extractorManipulator("lbc_left","gamma","gamma_left"))
rk.addManipulator("potentialExtLeft",sc.extractorManipulator("lbc_left","potential","potential_left"))
rk.addManipulator("tempExtLeft",sc.extractorManipulator("lbc_left","shTemp","temp_left"))

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
# Everything default except for more lenient absolute convergence tolerance
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["f"])

rk.addIntegrator("BE", integrator)


0 timestep to check analytic solution

In [ ]:
initialTimestep = 0.0

rk.setIntegratorGlobalData(1, 1, initialTimestep)


In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1", bdeStep.dict())


#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(1)
rk.setFixedStepOutput(1)


### Create config file

In [ ]:
rk.writeConfigFile()


### Data analysis


In [ ]:
numFiles = 1


#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]


In [ ]:
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames)
loadedData


Compare solution to analytic value. Note that $\gamma_e$ saved as modelbound data is calculated using the cut-off distribution temperature, so to rescale to unit temperature the analytical gamma must be divided by the sheath temperature variable temp_right.

In [ ]:
analyticGamma = (2-0.5*np.log(4*np.pi*elMass/ionMass))/loadedData["temp_right"].data[1,0]
abs(loadedData["gamma_right"].data[1,0] - analyticGamma)/analyticGamma

In [ ]:
nalyticGamma = (2-0.5*np.log(4*np.pi*elMass/ionMass))/loadedData["temp_left"].data[1,0]
abs(loadedData["gamma_left"].data[1,0] - analyticGamma)/analyticGamma